In [107]:
from rdflib import Graph, Literal, RDF, URIRef
# rdflib knows about quite a few popular namespaces, like W3C ontologies, schema.org etc.
import rdflib
import pandas as pd
from rdflib import Namespace

In [108]:
# ouvrir l'ontologie en tant que graphe avec rdflib

g = Graph() # création d'un graphe vide / remise à zéro
# On ouvre/parse le fichier de rdf avec toutes les classes de notre master ontologie 
# Le parse comme le html ta mère
g.parse("Ontologie_MasterTAL.rdf", format='xml')

# print(g.serialize(format='turtle')) # print en format turtle


# On crée un namespace (ex: foaf, rdf, rdfs, etc) pour notre ontologie
# Son URI est l'URI de notre ontologie
MT = Namespace("http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL#")
# On bind notre namespace à notre graphe pour dire il y a des trucs qu'on va mettre dedans
g.bind("mt", MT)      # bind a user-declared namespace to a prefix

# Ouvrir les dataframes 

In [109]:
formulaire = pd.read_csv("formulaire.csv")
df_etudiant = pd.read_csv("dataframe_eleves.csv")

In [110]:
dico_evaluation = {'DevoirMaisonGroupe': "Devoir maison (en groupe)", 
                   'DevoirTable': "Devoir sur table", 
                   'DevoirMaisonSolo': "Devoir maison (en solo)", 
                   'ProjetGroupe': 'Projet (en groupe)',
                   'ProjetSolo':'Projet (solo)',
                   'OralGroupe': 'Présentation orale (en groupe)',
                   'OralSolo': 'Présentation orale (solo)'
                   }
dico_reverse_eval = {v: k for k, v in dico_evaluation.items()}
print(dico_reverse_eval)


{'Devoir maison (en groupe)': 'DevoirMaisonGroupe', 'Devoir sur table': 'DevoirTable', 'Devoir maison (en solo)': 'DevoirMaisonSolo', 'Projet (en groupe)': 'ProjetGroupe', 'Projet (solo)': 'ProjetSolo', 'Présentation orale (en groupe)': 'OralGroupe', 'Présentation orale (solo)': 'OralSolo'}


In [111]:
dico_parcours = {"nom_parcours": ["M1_Nanterre_23-24", "M1_Inalco_23-24", "M1_Sorbonne_23-24", "M2_RD_Nanterre_24-25", "M2_RD_Inalco_24-25", "M2_RD_Sorbonne_24-25", "M2_Alterance_24-25", "M2_IM_24-25", "M2_TetraDom_24-25"],
                 "etablissement": ["Nanterre", "Inalco", "SorbonneNouvelle", "Nanterre", "Inalco", "SorbonneNouvelle", "Nanterre", "Inalco", "Inalco"],
                 "liste_eleves" : [[], [], [], [], [], [], [], [], []],
                 "annee" : ["2023-2024", "2023-2024", "2023-2024", "2024-2025", "2024-2025", "2024-2025", "2024-2025", "2024-2025", "2024-2025"],}


df_parcours = pd.DataFrame(dico_parcours)
df_parcours.to_csv("dataframe_parcours.csv", index=False)


In [112]:
def ajout_etudiant(df_etudiant, g, dico_reverse_eval):
    '''On ajoute les etudiants à notre ontologie'''
    for row_eleve in df_etudiant.itertuples(index=False, name = 'Eleve'):
        # On crée un URI pour chaque étudiant
        etudiant = URIRef(MT + (row_eleve.id).replace(" ", ""))
        # On ajoute l'étudiant à notre ontologie
        g.add((etudiant, RDF.type, MT.Etudiant))
        g.add((etudiant, MT.nom, Literal(row_eleve.nom)))
        g.add((etudiant, MT.prenom, Literal(row_eleve.prenom)))
        g.add((etudiant, MT.genre, Literal(row_eleve.genre)))
        g.add((etudiant, MT.dateNaissance, Literal(row_eleve.date_naissance)))
        g.add((etudiant, MT.nationalite, Literal(row_eleve.nationalite)))
        
        if row_eleve.redoublement == "non":
            g.add((etudiant, MT.redoublement, Literal(False)))
        else:
            g.add((etudiant, MT.redoublement, Literal(True)))
        
        g.add((etudiant, MT.tiersTemps, Literal(row_eleve.tiers_temps)))

        # Parcours M1 
        if "Nanterre" in row_eleve.inscription_M1:
            g.add((etudiant, MT.inscritA, URIRef(MT +"M1_Nanterre_23-24")))
        elif "INALCO" in row_eleve.inscription_M1:
            g.add((etudiant, MT.inscritA, URIRef(MT+"M1_Inalco_23-24")))
        elif "Sorbonne" in row_eleve.inscription_M1:
            g.add((etudiant, MT.inscritA, URIRef(MT+"M1_Sorbonne_23-24")))
        # Parcours M2
        if "Alternance" in row_eleve.parcours_M2: 
            g.add((etudiant, MT.inscritA, URIRef(MT+"M2_Alterance_24-25")))
        elif "Ingénierie Multilingue" in row_eleve.parcours_M2:
            g.add((etudiant, MT.inscritA, URIRef(MT+"M2_IM_24-25")))
        elif "TetraDom" in row_eleve.parcours_M2:
            g.add((etudiant, MT.inscritA, URIRef(MT+"M2_TetraDom_24-25")))
        elif "Recherche et Développement (Nanterre)" in row_eleve.parcours_M2:
            g.add((etudiant, MT.inscritA, URIRef(MT+"M2_RD_Nanterre_24-25")))
        elif "Recherche et Développement (INALCO)" in row_eleve.parcours_M2:
            g.add((etudiant, MT.inscritA, URIRef(MT+"M2_RD_Inalco_24-25")))
        elif "Recherche et Développement (Sorbonne Nouvelle)" in row_eleve.parcours_M2:
            g.add((etudiant, MT.inscritA, URIRef(MT+"M2_RD_Sorbonne_24-25")))

        if pd.notna(row_eleve.inscription_simultanee):
            g.add((etudiant, MT.inscritAussiA, Literal(row_eleve.inscription_simultanee)))
        if pd.notna(row_eleve.BAC):
            g.add((etudiant, MT.bac, Literal(row_eleve.BAC)))
        if pd.notna(row_eleve.licence):
            g.add((etudiant, MT.licence, Literal(row_eleve.licence)))
        if pd.notna(row_eleve.mention_licence):
            g.add((etudiant, MT.mention_licence, Literal(row_eleve.mention_licence)))
        if pd.notna(row_eleve.autre_master):
            g.add((etudiant, MT.autre_master, Literal(row_eleve.autre_master)))
        if pd.notna(row_eleve.mention_master_precedent):
            g.add((etudiant, MT.mention_master, Literal(row_eleve.mention_master_precedent)))
        if pd.notna(row_eleve.autres_diplomes):
            g.add((etudiant, MT.autre_diplome, Literal(row_eleve.autres_diplomes)))
        if pd.notna(row_eleve.job_etudiant):
            g.add((etudiant, MT.jobEtudiant, Literal(row_eleve.job_etudiant)))
        if pd.notna(row_eleve.profession_avant):
            g.add((etudiant, MT.profession_avant, Literal(row_eleve.profession_avant)))

        if pd.notna(row_eleve.prefere_sujet):
            g.add((etudiant, MT.sujet_pref, Literal(row_eleve.prefere_sujet)))
        
        # Boucle avec les types d'évaluations
        if pd.notna(row_eleve.prefere_evaluation):
            evals = (row_eleve.prefere_evaluation).split(', ')
            for eval in evals: 
                print(eval)
                if eval in dico_reverse_eval.keys():
                    g.add((etudiant, MT.prefereCommeEvaluation, URIRef(MT + dico_reverse_eval[eval])))
                else: 
                    g.add((etudiant, MT.prefereCommeEvaluation, URIRef(MT + (eval).replace(" ", "_"))))
                   
        # Boucle avec les noms des cours + formalisation de kk
        # g.add((etudiant, MT.prefereLeCoursDe, Literal(row_eleve.prefere_cours)))

        g.add((etudiant, MT.suggestion_maquette, Literal(row_eleve.suggestion_maquette)))
        
        # Attend formalisation des noms sur le formulaire
        # g.add((etudiant, MT.aCommeEnnemi, Literal(row_eleve.ennemi_jure)))
    
        g.add((etudiant, MT.sante_avant, Literal(row_eleve.sante_avant)))
        g.add((etudiant, MT.sante_apres, Literal(row_eleve.sante_apres)))
        g.add((etudiant, MT.quota_pleure, Literal(row_eleve.quota_pleure)))
        g.add((etudiant, MT.temps_sommeil, Literal(row_eleve.heures_sommeil)))
        
        g.add((etudiant, MT.difficulte_M1, Literal(row_eleve.difficulte_M1)))
    
        g.add((etudiant, MT.difficulte_M2, Literal(row_eleve.difficulte_alternance)))
        g.add((etudiant, MT.difficulte_M2, Literal(row_eleve.difficulte_IM)))
        g.add((etudiant, MT.difficulte_M2, Literal(row_eleve.difficulte_tetradom)))
        g.add((etudiant, MT.difficulte_M2, Literal(row_eleve.difficulte_RD)))

        if pd.notna(row_eleve.jeter_linux):
            g.add((etudiant, MT.relation_linux, Literal(row_eleve.jeter_linux))) 
        if pd.notna(row_eleve.jeter_mac):
            g.add((etudiant, MT.relation_mac, Literal(row_eleve.jeter_mac))) 

        if pd.notna(row_eleve.conseil_eleve):
            g.add((etudiant, MT.conseil_eleve, Literal(row_eleve.conseil_eleve)))

        if pd.notna(row_eleve.conseil_prof):
            g.add((etudiant, MT.conseil_prof, Literal(row_eleve.conseil_prof)))

        if pd.notna(row_eleve.recommandation):
            g.add((etudiant, MT.recommendation, Literal(row_eleve.recommandation)))

        


    print("--- printing mboxes ---")
    print(g.serialize(format='n3'))
    # print("--- printing xml format ---")
    # print(g.serialize(format='xml'))

    return g

ajout_etudiant(df_etudiant, g, dico_reverse_eval)

Devoir sur table
Projet (en groupe)
Présentation orale (en groupe)
Devoir maison (en groupe)
Devoir maison (en solo)
Projet (en groupe)
Projet (solo)
Devoir maison (en solo)
Projet (solo)
heu QCM?
Devoir maison (en groupe)
Devoir maison (en solo)
Projet (en groupe)
Projet (solo)
Devoir maison (en solo)
Projet (en groupe)
Projet (solo)
Devoir maison (en solo)
Projet (solo)
Devoir maison (en solo)
Projet (en groupe)
Projet (solo)

Devoir maison (en solo)
Projet (solo)
Devoir sur table
Devoir maison (en groupe)
Devoir maison (en solo)
Projet (en groupe)
Projet (solo)
Devoir maison (en solo)
Projet (solo)
Présentation orale (en groupe)
Présentation orale (solo)
Projet (solo)
Présentation orale (en groupe)
Présentation orale (solo)
Devoir maison (en solo)
Projet (en groupe)
Devoir maison (en solo)
Projet (en groupe)
Devoir maison (en solo)
Projet (en groupe)
Présentation orale (en groupe)
Devoir sur table
Devoir maison (en groupe)
Devoir maison (en solo)
Devoir maison (en solo)
Projet (en g

<Graph identifier=N4cffe66ab6484f94a6906a2bc8b3dab9 (<class 'rdflib.graph.Graph'>)>

In [116]:
def ajout_etablissement(g): 
    g.add((URIRef(MT+'Inalco'), RDF.type, MT.Etablissement))
    g.add((URIRef(MT+'Nanterre'), RDF.type, MT.Etablissement))
    g.add((URIRef(MT+'SorbonneNouvelle'), RDF.type, MT.Etablissement))
    return g

g = ajout_etablissement(g)
print(g.serialize(format='n3'))

@prefix mt: <http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL> a owl:Ontology .

mt:Alix_SIRVEN_VIÉNOT a mt:Etudiant ;
    mt:autre_master "MA in Applied Linguistics and Intercultural Studies" ;
    mt:bac "ES" ;
    mt:conseil_eleve "Reposez vous quand vous pouvez et surtout soyez bien entourés avec des gens qui comprennent ce que vous faites (c'est dur de faire un bide sur une super blague sur les couches de drop out de CNN) et dans les moments difficiles c'est important d'avoir des gens du master en soutien ce master est dure et ce seront vos piliers  " ;
    mt:conseil_prof "Soyez gentils on douille ici, demandez à vos étudiants si ils ont un tiers temps en début d'année on a trop de cours on oublie et après ça vous gonfle et ça stresse tout le

In [14]:
def ajout_prof(df_prof, g):
    '''On ajoute les profs à notre ontologie'''
    pass
	# print("--- printing mboxes ---")
	# print(g.serialize(format='n3'))

    # return g
    

In [9]:
def get_predicats(g):
	'''Choppe du prédicat pour voir les noms de relations utilisés dans l'ontologie
	et genre si il y en a qu'on a oublié'''
	predicats = []
	for s, p, o in g.triples((None, None, None)):
		predicats.append(p)
	return set(predicats)

print(get_predicats(g))

5


In [12]:
def get_classes(g):
	'''	Choppe tous les types de classes de onto '''
	classes = []
	for s, p, o in g.triples((None, RDF.type, None)):
		classes.append(o)
	return len(set(classes))

print(get_classes(g))

5
